In [ ]:
!pip install firecrawl-py

In [58]:
from firecrawl import FirecrawlApp
import re,json,time,os
#Put in your own key! not giving you mine
key = ''
app = FirecrawlApp(api_key=key)

In [59]:
def scrapeCard(url):
  response = app.scrape_url(url=url)
  lines = response['markdown'].split('\n')
  japanese_line = lines[3]
  japanese_pattern = r'<br>([^<]+)'
  japanese_matches = re.findall(japanese_pattern, japanese_line)
  japanese_text = " ".join(japanese_matches).strip()
  english_line = lines[4]
  english_pattern = r'\| ([^\|]+) \|'
  english_matches = re.findall(english_pattern, english_line)
  english_text = " ".join(english_matches).strip()
  return {"ja": japanese_text, "en": english_text}

In [64]:
def eatSet(start, size, setcode, delay=1, filename='card_data.json'):
    card_data = {}
    if os.path.exists(filename):
        with open(filename, 'r', encoding='utf-8') as json_file:
            card_data = json.load(json_file)

    for i in range(start, size):
        card_key = f'{setcode}{str(i).zfill(3)}'
        if card_key in card_data:
            continue
        url = f'https://www.heartofthecards.com/code/cardlist.html?card=WS_{setcode}{str(i).zfill(3)}&short=1'
        card_info = scrapeCard(url)
        card_data[card_key] = card_info

        # Save to JSON file incrementally and time.sleep as ratelimiting countermeasure
        with open(filename, 'w', encoding='utf-8') as json_file:
            json.dump(card_data, json_file, indent=4, ensure_ascii=False)
        time.sleep(delay)

In [68]:
#Usage of starting from the beginning of KGL(Kaguya-sama).
size = 104
setcode = "KGL/S79"
delay = 10
startindex = 1
#1 is starting index(if you're doing a set incrementally)
eatSet(startindex,size+1, setcode, delay)